In [ ]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd

import numpy as np
from scipy import stats

import ccdproc as ccdp
from ccdproc import ImageFileCollection

from astropy import units as u
from astropy.io import fits
from astropy.nddata import CCDData
from astropy.stats import mad_std
import astroalign as aa
# import PyQt6.QtWidgets as Qwid
# import PyQt6.QtCore as Qc

In [ ]:
# Модуль подготавливающий кадры к калибровке
                                                         
raw_files_path = Path('raw')                                        # путь (название папки) к нетронутым файлам
calibrated_files_path = Path('calibrated')                          # путь где будут находиться все файлы в правильном формате + мастер-кадры
raw_images = ccdp.ImageFileCollection(raw_files_path)
# raw_images.summary['file', 'imagetyp', 'filter', 'exptime', 'naxis1', 'naxis2']   # таблица данных об изображениях(для тестирования)
file_names = raw_images.files                                       # Список с наименованием файлов [list of str]
for i in range(len(file_names)):                                    # Цикл в котором: 1) в шапку записываются необходимые данные 2) меняется формат из .fit -> .fits
    data, header = fits.getdata(raw_files_path / file_names[i], header=True) 
    header['BUNIT'] = "adu"
    fits.writeto(calibrated_files_path / (file_names[i] +'s'), data, header, overwrite=True)
calibrated_images = ccdp.ImageFileCollection(calibrated_files_path)
for ccd, file_name in calibrated_images.ccds(return_fname=True):    # т-к отдельно снимаются и bias и dark кадры, то overscan просто обрезается(в данном цикле)
    ccd = ccdp.trim_image(ccd[:, :3339])                            # Trim 
    ccd.write(calibrated_files_path / file_name, overwrite=True)    # Сохранение результата


In [ ]:
#Bias Frame, Flat Field, Light Frame, Dark Frame(150sec)
# модуль создающий мастер-bias 

calibrated_biases = calibrated_images.files_filtered(imagetyp='Bias Frame', include_path=True) # collectin of bias-type files

combined_bias = ccdp.combine(calibrated_biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             mem_limit=350e6)                         
combined_bias.meta['combined'] = True
combined_bias.write(calibrated_files_path / 'masters' / 'combined_bias.fits', overwrite=True)
'''
# calibrated_images.summary['file', 'imagetyp', 'filter', 'exptime', 'naxis1', 'naxis2'] # таблица данных об изображениях(для тестирования)

dat = (fits.open(calibrated_files_path / 'masters' / 'combined_bias.fits'))[0].data     # подмодуль для просмотра combined_bias
fig = plt.figure(figsize=[10,8])

plt.imshow(dat, cmap = 'hot', norm = LogNorm())
cbar = plt.colorbar()
plt.show()
'''


In [ ]:
# модуль создающий master-dark для каждой группы с одинаковыми экспозициями (bias не вычитается из dark в данном случае)
darks = calibrated_images.summary['imagetyp'] == 'Dark Frame'
dark_times = set(calibrated_images.summary['exptime'][darks])
# print(dark_times)
for exp_time in sorted(dark_times):
    calibrated_darks = calibrated_images.files_filtered(imagetyp='Dark Frame', exptime=exp_time,
                                                     include_path=True)

    combined_dark = ccdp.combine(calibrated_darks,
                                 method='average',
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                                 mem_limit=350e6
                                )

    combined_dark.meta['combined'] = True

    dark_file_name = 'combined_dark_{:6.3f}.fits'.format(exp_time)
    combined_dark.write(calibrated_files_path / 'masters' / dark_file_name, overwrite=True)

In [ ]:
# модуль калибрующий(-bias) и комбинирующий flat

flat_files_names = calibrated_images.files_filtered(imagetyp='Flat Field')
for i in range(len(flat_files_names)):                                                                  # цикл вычитающий баяс из флэта
    a_flat = CCDData.read(calibrated_images.files_filtered(imagetyp='Flat Field', include_path=True)[i], unit='adu')
    calibrated_flats = ccdp.subtract_bias(a_flat, combined_bias)
    calibrated_flats.write(calibrated_files_path / 'flats' / flat_files_names[i], overwrite=True)


calibrated_flats_path = Path(calibrated_files_path / 'flats')
ifc = ccdp.ImageFileCollection(calibrated_flats_path)
flat_filters = set(h['filter'] for h in ifc.headers(imagetyp='Flat Field'))
# 'Red', 'Blue', 'Green'
def norm(a):
    m = np.median(a)
    return 1 / m

for filt in flat_filters:
    to_combine = ifc.files_filtered(imagetyp='Flat Field', filter=filt, include_path=True)
    combined_flat = ccdp.combine(to_combine,
                                 method='average', scale=norm,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                 mem_limit=350e6
                                )

    combined_flat.meta['combined'] = True
    flat_file_name = 'combined_flat_{}.fits'.format(filt.replace("''", "p"))
    combined_flat.write(calibrated_files_path / 'masters' / flat_file_name, overwrite=True)


'''
# подмодуль для просмотра combined_flat(последнего записанного)
dat = (fits.open(calibrated_files_path / 'masters' / flat_file_name))[0].data
fig = plt.figure(figsize=[10,8])
plt.imshow(dat)
cbar = plt.colorbar()
plt.show()
'''

In [ ]:
#Bias Frame, Flat Field, Light Frame, Dark Frame


def find_nearest_dark_exposure(image, dark_exposure_times, tolerance=0.5): # функция нахождения dark файла с экспозицией как у научного кадра(из примера)
    """
    Find the nearest exposure time of a dark frame to the exposure time of the image,
    raising an error if the difference in exposure time is more than tolerance.
    
    Parameters
    ----------
    
    image : astropy.nddata.CCDData
        Image for which a matching dark is needed.
    
    dark_exposure_times : list
        Exposure times for which there are darks.
    
    tolerance : float or ``None``, optional
        Maximum difference, in seconds, between the image and the closest dark. Set
        to ``None`` to skip the tolerance test.
    
    Returns
    -------
    
    float
        Closest dark exposure time to the image.
    """

    dark_exposures = np.array(list(dark_exposure_times))
    idx = np.argmin(np.abs(dark_exposures - image.header['exptime']))
    closest_dark_exposure = dark_exposures[idx]

    if (tolerance is not None and 
        np.abs(image.header['exptime'] - closest_dark_exposure) > tolerance):
        
        raise RuntimeError('Closest dark exposure time is {} for image of exposure '
                           'time {}.'.format(closest_dark_exposure, image.header['exptime']))
        
    
    return closest_dark_exposure


ifc = ccdp.ImageFileCollection(calibrated_files_path)
masters = ccdp.ImageFileCollection(calibrated_files_path / 'masters')
# lights = ifc.summary[ifc.summary['imagetyp'] == 'Light Frame']
exposure = 'exptime'

combined_darks = {ccd.header[exposure]: ccd for ccd in masters.ccds(imagetyp='Dark Frame', combined=True)}
combined_flats = {ccd.header['filter']: ccd for ccd in masters.ccds(imagetyp='Flat Field', combined=True)}

for light, file_name in ifc.ccds(imagetyp='Light Frame', return_fname=True, ccd_kwargs=dict(unit='adu')):
    
    closest_dark = find_nearest_dark_exposure(light, combined_darks.keys())
    reduced = ccdp.subtract_dark(light, combined_darks[closest_dark], exposure_time='exptime', exposure_unit=u.second)
    good_flat = combined_flats[reduced.header['filter']]
    reduced = ccdp.flat_correct(reduced, good_flat)
    reduced.write(Path('result') / file_name, overwrite=True)


In [ ]:
'''
#section for aligment data
dir = Path('result')               # way to fully calibrated files
imgs = ccdp.ImageFileCollection(dir)    # collection of all "result" files (only sourse imgs)
img_list = imgs.files                   # list of str (file names)
n = 0
# ref_image = img_list[n]         
# images_to_align = img_list[1:]  # все остальные элементы котороые выравниваются по элементу n [название файлов списком]
ref_data = CCDData.read(dir / img_list[n]) # элемент номер n (по которому будет проходить align)
target = (fits.open(dir / img_list[n], do_not_scale_image_data=True)[0].data).newbyteorder()
for i in range (len(img_list)):
    if i == n:
        ref_data.write(Path('aligned') / img_list[i], overwrite=True)
    else:
        data_to_align = fits.open(dir / img_list[i])[0].data
        source = data_to_align.newbyteorder()
        img_aligned, footprint = aa.register(source, target, detection_sigma=2)
        print(n)
        img_aligned.write(Path('aligned') / img_list[i], overwrite=True)



print('debug')
'''